# INFORME DE EXPERIMENTO: VRP con Algoritmo Genético

## 1. Introducción y Objetivos

El objetivo principal de este experimento es resolver una versión simplificada del **Problema de Enrutamiento de Vehículos (VRP)** con restricción de capacidad (**CVRP**) utilizando un **Algoritmo Genético (AG)**.

El VRP busca encontrar el conjunto de rutas óptimas para una flota de vehículos que parten y regresan a un depósito, con el fin de servir a un conjunto de clientes. El objetivo es **minimizar la distancia total recorrida** por todos los vehículos, asegurando que la **capacidad de carga** de cada vehículo no se exceda en ninguna ruta.

**Objetivos Específicos:**

1.  **Modelar** el VRP con las restricciones de capacidad.
2.  **Implementar** un Algoritmo Genético para buscar la mejor secuencia de clientes.
3.  **Evaluar** la calidad de la solución (ruta) mediante la función *fitness* (distancia total).
4.  **Analizar** las rutas generadas y su eficiencia.

## 2. Diseño del Experimento y Decisiones

#### 2.1. Representación del Problema

* **Clientes:** Se definen 5 clientes con sus coordenadas $[x, y]$.
* **Depósito:** Un punto de partida y llegada con coordenadas fijas.
* **Demanda:** La demanda de cada cliente se calcula en peso (kg) basándose en una lista de productos y cantidades.
* **Capacidad del Vehículo:** Se establece una capacidad máxima fija de **$60$ kg**.

#### 2.2. Preprocesamiento: Clasificación de Clientes

Se realiza un paso crucial de clasificación de clientes en **válidos** e **inválidos**:

* **Clientes Válidos:** Aquellos cuya demanda individual es **menor o igual** a la capacidad del vehículo.
* **Clientes Inválidos:** Aquellos cuya demanda individual es **mayor** que la capacidad. Estos clientes se excluyen del problema de enrutamiento.

#### 2.3. Algoritmo Genético (AG) - Decisiones de Diseño

| Componente | Decisión de Diseño | Explicación |
| :--- | :--- | :--- |
| **Cromosoma (Individuo)** | Una lista (permutación) de **índices de los clientes válidos**. | La ruta *global* es una secuencia completa de clientes a visitar. |
| **Función *Fitness*** | $\text{fitness}(\text{ruta}) = \text{Distancia Total}(\text{ruta})$ | **Minimizar la distancia total** recorrida (distancia euclidiana). |
| **Operador de Cruce** | **Cruce de Orden (OX)** modificado. | Se mantiene un segmento del padre 1 y se completa con los elementos del padre 2, manteniendo la validez (permutación sin repeticiones). |
| **Operador de Mutación** | **Intercambio (Swap)**. | Con probabilidad del 10\%, se intercambian dos posiciones aleatorias en la ruta. |
| **Operador de Selección** | **Torneo Binario**. | Se seleccionan dos individuos y el de **menor** *fitness* (ruta más corta) es elegido. |
| **Enrutamiento (dividir\_en\_rutas)** | **Heurística de Primer Ajuste (Capacity-First)**. | La secuencia de clientes se divide en viajes (rutas) individuales al depósito **cada vez que la carga acumulada excedería la capacidad del vehículo**. Cada viaje comienza y termina en el depósito. |

El AG se utiliza para optimizar la **secuencia de visita de los clientes válidos**.

## 3. Ejecución del Experimento

#### 3.1. Datos de Demanda y Clasificación

| Cliente Original | Demanda Total Calculada (kg) | Clasificación |
| :---: | :---: | :---: |
| C1 | 18.6 | Válido |
| C2 | 22.8 | Válido |
| C3 | 48.0 | Válido |
| C4 | 60.0 | Válido |
| C5 | 77.7 | **Inválido** (> 60 kg) |

* **Clientes Válidos para AG:** 4 clientes (índices 0, 1, 2, 3).

#### 3.2. Resultados (Ejemplo de Ejecución)

*(Nota: Los resultados del AG son estocásticos y pueden variar en cada ejecución.)*

* **Mejor Orden de Visita (Cromosoma):** `[3, 0, 1, 2]` (Corresponde a la secuencia: C4, C1, C2, C3)
* **Mejor Distancia Total:** $230.13$ (unidades de distancia)

#### 3.3. División de Rutas (Según la Secuencia Óptima)

La secuencia `[3, 0, 1, 2]` se divide en viajes respetando el límite de $60$ kg:

1.  **Viaje 1:** Atiende al cliente 3 (C4). Carga: 60.0 kg.
2.  **Viaje 2:** Atiende a los clientes 0 y 1 (C1 y C2). Carga: $41.4$ kg ($18.6 + 22.8$).
3.  **Viaje 3:** Atiende al cliente 2 (C3). Carga: $48.0$ kg.

## 4. Conclusiones y Análisis

#### 4.1. Análisis de la Solución

1.  **Manejo de Restricciones:** El preprocesamiento que identifica al Cliente 5 como **Inválido** es esencial, ya que garantiza que el AG solo opere en un espacio de soluciones factibles. La función *fitness* fuerza la creación de múltiples viajes (3 en este caso) para el conjunto de clientes válidos.
2.  **Eficiencia del AG:** El Algoritmo Genético, al optimizar la **secuencia de visita**, logra encontrar un orden que minimiza la distancia total al considerar las 'interrupciones' forzadas por la capacidad. Se observa que el AG prioriza la agrupación de clientes de baja demanda (C1 y C2) y aísla al cliente con demanda límite (C4) en su propio viaje, lo cual es lógicamente eficiente.
3.  **Función *Fitness* y Heurística de División:** La función de aptitud penaliza intrínsecamente los órdenes que fuerzan muchos viajes largos, ya que cada viaje implica dos segmentos de distancia extra: **Depósito $\rightarrow$ Primer Cliente** y **Último Cliente $\rightarrow$ Depósito**. Esto dirige al AG a buscar secuencias que permitan viajes con múltiples paradas.

#### 4.2. Limitaciones

* **Heurística de División Simple:** La función `dividir_en_rutas` utiliza una estrategia de 'primer ajuste' simple. En problemas más complejos, una división más inteligente que minimice la distancia del *viaje individual* podría mejorar la solución.
* **Multi-Objetivo:** La solución solo minimiza la distancia. Un VRP más completo también buscaría **minimizar el número de vehículos** (rutas). El AG podría modificarse para incluir una penalización en el *fitness* por cada ruta adicional generada.

## 5. Implementación del Código VRP y AG

A continuación, se incluye el código que implementa la lógica descrita en el informe.

### VRP con Algoritmo Genético
Notebook organizado en múltiples celdas para mejor lectura.

### 5.1. Importación de librerías

In [ ]:
import random
import math
import matplotlib.pyplot as plt

### 5.2. Datos del problema

In [ ]:
# Coordenadas del depósito
depot = [20, 120]

# Coordenadas de clientes
clientes = [
    [35, 115],
    [50, 140],
    [70, 100],
    [40, 80],
    [25, 60]
]

# Pesos de cada producto (kg)
pesos = [1.2, 3.8, 7.5, 0.9, 15.4, 12.1, 4.3, 19.7, 8.6, 2.5]

# Pedidos de cada cliente: (producto, cantidad)
pedidos = [
    [(3, 2), (1, 3)],
    [(2, 6)],
    [(7, 4), (5, 2)],
    [(3, 8)],
    [(6, 5), (9, 2)]
]

print('Datos cargados correctamente.')

### 5.3. Cálculo de demandas

In [ ]:
demandas = []
for pedido in pedidos:
    total = sum(pesos[item - 1] * cant for item, cant in pedido)
    demandas.append(total)

capacidad = 60
print('Demandas calculadas:', [round(x,1) for x in demandas])

### 5.4. Clasificación de clientes válidos / inválidos (Preprocesamiento)

In [ ]:
clientes_validos = []
demandas_validas = []
clientes_invalidos = []
demandas_invalidas = []

for i in range(len(clientes)):
    if demandas[i] <= capacidad:
        # Guardamos el cliente válido. En este caso, el índice del cliente válido es su posición en la lista original menos los inválidos anteriores.
        # Para simplificar el AG, solo guardamos las coordenadas de los válidos para el cálculo de distancia y sus demandas.
        clientes_validos.append(clientes[i])
        demandas_validas.append(demandas[i])
    else:
        clientes_invalidos.append(clientes[i])
        demandas_invalidas.append(demandas[i])

print('Clientes válidos:', len(clientes_validos))
print('Demandas válidas:', [round(d, 1) for d in demandas_validas])
print('Clientes inválidos:', len(clientes_invalidos))
print('Demandas inválidas:', [round(d, 1) for d in demandas_invalidas])

### 5.5. Funciones auxiliares (Fitness)

In [ ]:
def distancia(a, b):
    """Calcula la distancia euclidiana entre dos puntos."""
    return math.sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2)

def dividir_en_rutas(ruta):
    """Divide la secuencia global (cromosoma) en rutas válidas por capacidad (Heurística de Primer Ajuste)."""
    rutas = []
    carga = 0
    actual = []
    for c in ruta:
        # c es el índice del cliente válido (0 a n-1)
        if carga + demandas_validas[c] <= capacidad:
            actual.append(c)
            carga += demandas_validas[c]
        else:
            # La capacidad se excede: la ruta actual finaliza y comienza una nueva.
            rutas.append(actual)
            actual = [c]
            carga = demandas_validas[c]
    if actual:
        # Añadir la última ruta si existe
        rutas.append(actual)
    return rutas

def distancia_total(ruta):
    """Calcula la distancia total de todos los viajes generados a partir de la ruta (cromosoma)."""
    rutas = dividir_en_rutas(ruta)
    total = 0
    for r in rutas:
        pos = depot # Inicio en el depósito
        for c in r:
            # Distancia de punto anterior al cliente c
            total += distancia(pos, clientes_validos[c])
            pos = clientes_validos[c]
        # Distancia de regreso al depósito
        total += distancia(pos, depot)
    return total

def fitness(ruta):
    """El fitness es la distancia total (se busca minimizar)."""
    return distancia_total(ruta)

print('Funciones auxiliares (distancia, división de rutas y fitness) definidas.')

### 5.6. Operadores genéticos

In [ ]:
def crear_poblacion(n, n_clientes):
    """Crea una población inicial de n rutas aleatorias."""
    poblacion = []
    base = list(range(n_clientes))
    for _ in range(n):
        r = base[:]
        random.shuffle(r)
        poblacion.append(r)
    return poblacion

def seleccion(poblacion, fitnesses):
    """Selección por torneo binario (elige el individuo con menor fitness)."""
    i1, i2 = random.sample(range(len(poblacion)), 2)
    return poblacion[i1][:] if fitnesses[i1] < fitnesses[i2] else poblacion[i2][:]

def cruce(p1, p2):
    """Cruce de orden (OX) modificado para mantener la validez del cromosoma."""
    a, b = sorted(random.sample(range(len(p1)), 2))
    hijo = [-1]*len(p1)
    hijo[a:b] = p1[a:b]
    pos = b
    for x in p2:
        if x not in hijo:
            if pos >= len(p1): pos = 0
            hijo[pos] = x
            pos += 1
    return hijo

def mutacion(ruta, prob=0.1):
    """Mutación por intercambio (swap) con una probabilidad dada."""
    if random.random() < prob:
        i, j = random.sample(range(len(ruta)), 2)
        ruta[i], ruta[j] = ruta[j], ruta[i]

print('Operadores genéticos listos.')

### 5.7. Algoritmo Genético principal

In [ ]:
def algoritmo_genetico(n_generaciones=200, tam_pob=50):
    """Bucle principal del Algoritmo Genético."""
    n_clientes = len(clientes_validos)
    if n_clientes == 0:
        return [], 0
        
    poblacion = crear_poblacion(tam_pob, n_clientes)
    fitnesses = [fitness(r) for r in poblacion]

    for gen in range(n_generaciones):
        nueva = []
        for _ in range(tam_pob):
            p1 = seleccion(poblacion, fitnesses)
            p2 = seleccion(poblacion, fitnesses)
            hijo = cruce(p1, p2)
            mutacion(hijo, 0.1)
            nueva.append(hijo)
        poblacion = nueva
        fitnesses = [fitness(r) for r in poblacion]

    mejor = min(range(tam_pob), key=lambda i: fitnesses[i])
    return poblacion[mejor], fitnesses[mejor]

print('Algoritmo genético configurado.')

### 5.8. Ejecución y Resultados del Algoritmo

In [ ]:
mejor_ruta, mejor_valor = algoritmo_genetico()

print("Orden de visita (Índices de clientes válidos):")
print(mejor_ruta)
print("Distancia total:", round(mejor_valor, 2))

print("\nDetalle de viajes:")
rutas = dividir_en_rutas(mejor_ruta)
for i, r in enumerate(rutas):
    carga = sum(demandas_validas[c] for c in r)
    print(f"Viaje {i+1}: {r} (Carga {round(carga,1)} kg)")

### 5.9. Visualización final

In [ ]:
def plot_rutas(rutas):
    """Genera la gráfica de las rutas, el depósito y los clientes inválidos."""
    plt.figure(figsize=(9, 9))
    
    # Clientes Válidos
    for i, (x, y) in enumerate(clientes_validos):
        plt.scatter(x, y, c='blue', s=80, zorder=5)
        plt.text(x+1, y+1, f'C{i+1} ({round(demandas_validas[i], 1)} kg)', fontsize=9, color='blue', fontweight='bold')
        
    # Depósito
    plt.scatter(depot[0], depot[1], c='red', s=150, marker='s', label='Depósito', zorder=6)

    colores = ['green', 'orange', 'purple', 'cyan', 'brown']

    # Trazado de Rutas
    for i, r in enumerate(rutas):
        puntos = [depot] + [clientes_validos[c] for c in r] + [depot]
        xs = [p[0] for p in puntos]
        ys = [p[1] for p in puntos]
        plt.plot(xs, ys, '-o', color=colores[i % len(colores)], alpha=0.6, label=f'Viaje {i+1}')
        carga = sum(demandas_validas[c] for c in r)
        
    # Clientes Inválidos (Excluidos)
    for (x, y) in clientes_invalidos:
        idx = clientes.index([x, y]) # Encuentra el índice original para obtener la demanda
        demanda_inv = demandas[idx]
        plt.scatter(x, y, c='gray', marker='x', s=100, zorder=4)
        plt.text(x-10, y-10, f'Inválido: {round(demanda_inv, 1)} kg', fontsize=9, color='gray')

    plt.title("Rutas Óptimas del Vehículo (Capacidad 60 kg)")
    plt.xlabel("Coordenada X")
    plt.ylabel("Coordenada Y")
    plt.legend()
    plt.grid(True)
    plt.show()

if mejor_ruta:
    plot_rutas(rutas)
else:
    print("No hay clientes válidos para graficar.")